In [2]:
# This file is for training 20 child classifers with parent trained to do parent classification only and saving their dictionaries

In [13]:
# This is mapping between fine label, ranging from 0 to 99, to labels ranging from 0 to 4
# key: parent category -> value: dictionary {key: fine label 0-99 -> value: label 0-4}
import json
f = open('../parent_to_child_class_from_0_to_5.json')
parent_to_child_class_from_0_to_5 = json.load(f)
print(parent_to_child_class_from_0_to_5)

{'11': {'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}, '15': {'27': 0, '29': 1, '44': 2, '78': 3, '93': 4}, '4': {'0': 0, '51': 1, '53': 2, '57': 3, '83': 4}, '14': {'2': 0, '11': 1, '35': 2, '46': 3, '98': 4}, '1': {'1': 0, '32': 1, '67': 2, '73': 3, '91': 4}, '5': {'22': 0, '39': 1, '40': 2, '86': 3, '87': 4}, '18': {'8': 0, '13': 1, '48': 2, '58': 3, '90': 4}, '3': {'9': 0, '10': 1, '16': 2, '28': 3, '61': 4}, '10': {'23': 0, '33': 1, '49': 2, '60': 3, '71': 4}, '17': {'47': 0, '52': 1, '56': 2, '59': 3, '96': 4}, '2': {'54': 0, '62': 1, '70': 2, '82': 3, '92': 4}, '9': {'12': 0, '17': 1, '37': 2, '68': 3, '76': 4}, '8': {'3': 0, '42': 1, '43': 2, '88': 3, '97': 4}, '16': {'36': 0, '50': 1, '65': 2, '74': 3, '80': 4}, '6': {'5': 0, '20': 1, '25': 2, '84': 3, '94': 4}, '12': {'34': 0, '63': 1, '64': 2, '66': 3, '75': 4}, '19': {'41': 0, '69': 1, '81': 2, '85': 3, '89': 4}, '7': {'6': 0, '7': 1, '14': 2, '18': 3, '24': 4}, '13': {'26': 0, '45': 1, '77': 2, '79': 3, '99': 4}, '0': {'4':

In [14]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
import torch
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import torchvision.transforms as tt
from torchvision import datasets, transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.metrics import *
from torchsummary import summary

class CIFAR10(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    base_folder = 'cifar-10-batches-py'
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    meta = {
        "filename": "batches.meta",
        "key": "label_names",
        "md5": "5ff9c542aee3614f3951f8cda6e48888",
    }


    def __init__(self, root, train=True,
                 transform=None, target_transform=None,
                 download=False, coarse=False, coarseNumber=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.coarse = coarse

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.train:
            self.train_data = []
            self.train_labels = []
            self.train_coarse_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                    if self.coarse:
                        self.train_coarse_labels += entry['coarse_labels']
                fo.close()
            new_train_coarse_labels=[]
            new_train_fine_labels=[]
            new_train_data=[]
            for data, fine,coarse in zip(self.train_data[0],self.train_labels, self.train_coarse_labels):
                if coarse==coarseNumber:
                    new_train_data.append(data)
                    new_train_fine_labels.append(parent_to_child_class_from_0_to_5[str(coarseNumber)][str(fine)])
                    new_train_coarse_labels.append(coarseNumber)

            self.train_data=np.array([np.array(xi) for xi in new_train_data])
            self.train_labels=new_train_fine_labels
            self.train_coarse_labels=new_train_coarse_labels

            self.train_data = self.train_data.reshape((len(new_train_data), 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
        else:
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']

            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
                if self.coarse:
                    self.test_coarse_labels = entry['coarse_labels']
                new_test_coarse_labels=[]
                new_test_fine_labels=[]
                new_test_data=[]

                for data, fine,coarse in zip(self.test_data,self.test_labels, self.test_coarse_labels):
                    if coarse==coarseNumber:
                        new_test_data.append(data)
                        new_test_fine_labels.append(parent_to_child_class_from_0_to_5[str(coarseNumber)][str(fine)])
                        new_test_coarse_labels.append(coarseNumber)

                self.test_data=np.array([np.array(xi) for xi in new_test_data])

                self.test_labels=new_test_fine_labels
                self.test_coarse_labels=new_test_coarse_labels
            fo.close()
            self.test_data = self.test_data.reshape((len(new_test_data), 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
            if self.coarse:
                coarse_target = self.train_coarse_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]
            if self.coarse:
                coarse_target = self.test_coarse_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        if not self.coarse:
            return img, target
        else:
            return img, target, coarse_target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)


class CIFAR100(CIFAR10):
    """`CIFAR100 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    This is a subclass of the `CIFAR10` Dataset.
    """
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]


In [15]:
transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         #tt.Normalize(mean,std,inplace=True) 
                         ]
                         )
transform_test = tt.Compose([tt.ToTensor(), 
                             #tt.Normalize(mean,std)
                             ])

In [16]:
batch_size = 400
epochs = 120
max_lr = 0.001
grad_clip = 0.01
weight_decay =0.001
opt_func = torch.optim.Adam

In [17]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [18]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def __init__(self, fine):
        super(ImageClassificationBase, self).__init__()
        self.fine = fine
        #self.parent_to_child_class={}
        self.unique_outs=[]
    
    def training_step(self, batch):
        images, fine, coarse = batch
        if self.fine:
            labels=fine
        else:
            labels=coarse
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, fine, coarse = batch 

 
        if self.fine:
            labels=fine
        else:
            labels=coarse
        out = self(images)

        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
        
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes, fine):
        super().__init__(fine)
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True) 
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128)) 
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True) 
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512)) 
        self.conv5 = conv_block(512, 1028, pool=True) 
        self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))  
        
        # self.classifier_parent = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
        #                                 nn.Flatten(), # 1028 
        #                                 nn.Linear(1028, 20))
        
        # self.classifier_child = self.classifier_parent

        self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                nn.Flatten(), # 1028 
                                nn.Linear(1028, num_classes)
                                )

        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        out = self.classifier(out)
        #out = self.classifier_child(out)

        return out

In [19]:
@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, test_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [20]:
# Generate testing accuracy, predicted label, confusion matrix, and table for classification report
def test_label_predictions(model, device, test_loader, fine):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        if fine:
            for data, target, _ in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)
        else:
            for data, _, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)

    return [i.item() for i in actuals], [i.item() for i in predictions]

In [22]:
accuracy_list=[]

for i in range(20):
    train_data = CIFAR100('./data', train=True,
                 transform=transform_train,
                 download=True, coarse=True, coarseNumber=i)
    test_data = CIFAR100('./data', train=False,
                    transform=transform_test,
                    download=True, coarse=True, coarseNumber=i)

    train_length = train_data.__len__() # Length training dataset
    train_indices = np.arange(train_length)
        
    train_loader = torch.utils.data.DataLoader(
                            train_data, 
                            batch_size=batch_size, 
                            num_workers=2,
                            pin_memory=True,
                        )

    test_loader = torch.utils.data.DataLoader(
                            test_data, 
                            batch_size=batch_size*2,
                            num_workers=2,
                            pin_memory=True
                        )

    device = get_default_device()
    trainloader = DeviceDataLoader(train_loader, device)
    testloader = DeviceDataLoader(test_loader, device)


    model20To100 = ResNet9(3, 20, True)
    model20To100.load_state_dict(torch.load('../group_1028_to_parent_Loss_parent_only_pretrained_model.h5'))
    for param in model20To100.parameters():
        param.requires_grad=False

    model20To100.classifier= nn.Sequential(model20To100.classifier,
                                        nn.ReLU(),
                                        nn.Linear(20, 5),
                                        )
                                            
    model20To100=to_device(model20To100, device)
    history = [evaluate(model20To100, testloader)]
    print(history)
    history += fit_one_cycle(50, max_lr*4, model20To100, trainloader, testloader, 
                                grad_clip=grad_clip, 
                                weight_decay=weight_decay, 
                                opt_func=opt_func)
    
    y_test, y_pred = test_label_predictions(model20To100, device, testloader, True)
    ac=accuracy_score(y_test, y_pred)
    print('!!! Accuracy score: %f' % ac)
    accuracy_list.append(ac)

    #torch.save(model20To100.state_dict(), 'group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}.h5'.format(i))

Files already downloaded and verified
Files already downloaded and verified
[{'val_loss': 13.137938499450684, 'val_acc': 0.2160000056028366}]
Epoch [0], last_lr: 0.00019, train_loss: 17.9498, val_loss: 14.8632, val_acc: 0.2080
Epoch [1], last_lr: 0.00031, train_loss: 17.2424, val_loss: 15.7582, val_acc: 0.2100
Epoch [2], last_lr: 0.00050, train_loss: 16.1065, val_loss: 15.4090, val_acc: 0.2140
Epoch [3], last_lr: 0.00076, train_loss: 14.4850, val_loss: 13.9833, val_acc: 0.2260
Epoch [4], last_lr: 0.00109, train_loss: 12.4904, val_loss: 11.8318, val_acc: 0.2180
Epoch [5], last_lr: 0.00145, train_loss: 10.0801, val_loss: 8.8206, val_acc: 0.1960
Epoch [6], last_lr: 0.00185, train_loss: 7.0247, val_loss: 5.4656, val_acc: 0.2240
Epoch [7], last_lr: 0.00225, train_loss: 4.8046, val_loss: 4.0087, val_acc: 0.2280
Epoch [8], last_lr: 0.00265, train_loss: 3.4494, val_loss: 3.4152, val_acc: 0.1740
Epoch [9], last_lr: 0.00302, train_loss: 3.1481, val_loss: 3.1516, val_acc: 0.1940
Epoch [10], last_

In [23]:
accuracy_list

[0.392,
 0.516,
 0.392,
 0.504,
 0.492,
 0.536,
 0.414,
 0.43,
 0.452,
 0.486,
 0.504,
 0.436,
 0.422,
 0.526,
 0.33,
 0.484,
 0.362,
 0.41,
 0.5,
 0.592]

In [25]:
print("average child classification accuracy with embedding suited only for parent classification: ", sum(accuracy_list)/len(accuracy_list))

average child classification accuracy with embedding suited only for parent classification:  0.45899999999999996
